In [ ]:
import torch
import copy
import numpy as np

In [ ]:
# n_samples_list = [50000, 100000, 200000]
n_samples_list = [1000000, 2500000, 10000000]
gamma_th = 5/3
K = [8.9493e-02, None, None, None]  
Gamma = [1.3569e+00, 3.0050e+00, 2.9880e+00, 2.8510e+00]
rho_breaks = [2.3674e-04, 8.1147e-04, 1.6191e-03]

rho_min = 2e-5
rho_max = 2e-3

loaded_stats_out = np.loadtxt("./speed_test/gpu/mean_std_out_s_pen100.txt")
train_out_mean = torch.tensor(loaded_stats_out[0], dtype=torch.float32)
train_out_std = torch.tensor(loaded_stats_out[1], dtype=torch.float32)

loaded_stats_in = np.loadtxt("./speed_test/gpu/mean_std_in_s_pen100.txt")
train_in_mean = torch.tensor(loaded_stats_in[0], dtype=torch.float32)
train_in_std = torch.tensor(loaded_stats_in[1], dtype=torch.float32)

for n_samples in n_samples_list:
    rho = rho_min + (rho_max - rho_min) * torch.rand(n_samples, 1)
    vx  = 0.721 * torch.rand(n_samples, 1)
    W = (1 - vx**2).pow(-0.5)

    a_i = torch.zeros(len(K))
    for i in range(1, len(K)):
        K[i] = K[i-1] * rho_breaks[i-1]**(Gamma[i-1] - Gamma[i])
        a_i[i] = a_i[i-1] + K[i-1] * rho_breaks[i-1]**(Gamma[i-1] - 1) / (Gamma[i-1] - 1) - K[i] * rho_breaks[i-1]**(Gamma[i] - 1) / (Gamma[i] - 1)

    p = torch.zeros_like(rho)
    eps_cold = torch.zeros_like(rho)
    h = torch.zeros_like(rho)
    eps = torch.zeros_like(rho)
    for i in range(len(K)):
        if i == 0:
            mask = rho < rho_breaks[i]
        elif i < len(K) - 1:
            mask = (rho >= rho_breaks[i-1]) & (rho < rho_breaks[i])
        else:
            mask = rho >= rho_breaks[i-1]
    
        eps_cold[mask] = a_i[i] + K[i] * rho[mask]**(Gamma[i] - 1) / (Gamma[i] - 1)
        p[mask] = K[i] * rho[mask]**Gamma[i]

    eps_min = 0
    eps_max = 2
    eps_th = eps_min + (eps_max - eps_min) * torch.rand(n_samples, 1)
    p_th = (gamma_th - 1) * rho * eps_th
    p += p_th
    h = 1 + eps_cold + eps_th + p / rho

    D = rho * W
    Sx = rho * h * W**2 * vx
    tau = rho * h * W**2 - p - D

    inputs = torch.cat((D, Sx, tau), dim=1)
    outputs = copy.deepcopy(p)

    inputs_scaled = (inputs - train_in_mean) / train_in_std
    outputs_scaled = (outputs - train_out_mean) / train_out_std
    
    inputs_unscaled_np = inputs.numpy()
    outputs_unscaled_np = outputs.numpy()
    inputs_scaled_np = inputs_scaled.numpy()
    outputs_scaled_np = outputs_scaled.numpy()
    
    np.savetxt("inputs_test_unscaled_" + str(n_samples) + ".txt", inputs_unscaled_np, fmt='%.9g')
    np.savetxt("outputs_test_unscaled_" + str(n_samples) + ".txt", outputs_unscaled_np, fmt='%.9g')
    np.savetxt("inputs_test_scaled_" + str(n_samples) + ".txt", inputs_scaled_np, fmt='%.9g')
    np.savetxt("outputs_test_scaled_" + str(n_samples) + ".txt", outputs_scaled_np, fmt='%.9g')


In [ ]:
# Quick sanity check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "NNC2PS.pth"
model = torch.jit.load(model_path, map_location=device)
model = model.to(device)
model.eval()

input_values = np.loadtxt("inputs_test_scaled_200000.txt")

input_values = (input_values - train_in_mean.cpu().numpy()) / train_in_std.cpu().numpy() # Comment out to check whether scaling is done expectedly
with torch.no_grad():
    # input_values = input_scaler.transform(input_values)
    input_values = torch.from_numpy(input_values).float()
    predictions = model(input_values.to(device))
    predictions = predictions.cpu().numpy()

In [ ]:
predictions * train_out_std.cpu().numpy() + train_out_mean.cpu().numpy()

In [ ]:
predictions * train_out_std.cpu().numpy() + train_out_mean.cpu().numpy() # Run again after commenting out input_values scaling two cells above